In [34]:
import pandas as pd
import numpy as np
import re
import os
import lawrouge

In [35]:
# 读取Excel文件
df_real = pd.read_excel('real_order_14june.xlsx')

folder_path = './text5'

c:\Python312\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [40]:
def contains_number(s):
    if re.search(r'\d', s):
        return True
    return False

def find_split_code(chars, mixed_string):
    # 使用正则表达式匹配数字和非数字字符：当数字和字母都出现时才判断是TRUE
    digits = re.findall(r'\d+', mixed_string)
    letters = re.findall(r'\D+', mixed_string)
    letters_lower = [x.lower() for x in letters]
    letters_upper = [x.upper() for x in letters]
    res = digits+letters_lower+letters_upper
    res = [x.strip() for x in res]
    ans = 0 # 记录找到的字符个数
    for item in res:
        if item in chars:
            ans += 1
    return ans==len(digits)+len(letters)

def sliding_window(chars, window_size, course_code):
    # 滑动窗口寻找
    ans = False 
    # print(course_code,chars)
    if course_code in chars:
        return True
    if course_code.replace(" ", "") in chars:
        return True
    for i in range(len(chars) - window_size + 1):
        ans = find_split_code(chars[i:i + window_size], course_code)
        if ans:
            break
    return ans

def check_nan(course_code, summary):
    chars = summary.replace(" ", "")
    # chars = summary.split()
    return sliding_window(chars, 10, course_code)

def search_file_by_prefix(folder_path, file_prefix):
    # 列出文件夹中的所有文件
    all_content = ''
    for filename in os.listdir(folder_path):
        # 检查文件名是否以给定的前缀开头
        if filename.startswith(file_prefix):
            # 如果是，打印文件名和内容
            # print(f"找到文件: {filename}")
            # 读取文件内容
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                content = file.read()
            # return content
            all_content += content
    if all_content:
        return all_content
    else:
        # 如果文件没有找到，打印一条消息
        print(f"没有找到以{file_prefix}开头的文件。")
        temp.append(file_prefix)
        return None

In [41]:
df_real['course_code_valid'] = np.nan
select_df1 = df_real[(pd.isna(df_real['course_code'])==False) & (pd.isna(df_real['file_id'])==False)]

In [28]:
# row = select_df1.iloc[-1,:]
# file_id = int(row['file_id'])
# order_id = int(row['order_id'])
# course_code = row['course_code']
# content = search_file_by_prefix(folder_path, str(order_id)+'-'+str(file_id))
# check_nan(course_code, content)

True

In [42]:
temp = [] # 记录txt缺失
for index, row in select_df1.iterrows():
    file_id = int(row['file_id'])
    order_id = int(row['order_id'])
    course_code = row['course_code']
    content = search_file_by_prefix(folder_path, str(order_id)+'-'+str(file_id))
    if content is None:
        df_real.loc[index, 'course_code_valid'] = 'missing'
    elif not content:
        df_real.loc[index, 'course_code_valid'] = 'blank'
    else:
        df_real.loc[index, 'course_code_valid'] = check_nan(course_code, content)

C:\Users\nlp\AppData\Local\Temp\ipykernel_21640\3565708192.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_real.loc[index, 'course_code_valid'] = check_nan(course_code, content)


没有找到以537814-3816578开头的文件。
没有找到以537814-3816580开头的文件。
没有找到以537804-3816415开头的文件。
没有找到以537804-3816419开头的文件。
没有找到以537573-3814604开头的文件。
没有找到以537573-3814607开头的文件。
没有找到以537573-3814608开头的文件。
没有找到以537573-3814609开头的文件。
没有找到以537573-3814610开头的文件。
没有找到以537573-3814611开头的文件。
没有找到以537573-3814612开头的文件。
没有找到以537396-3813016开头的文件。
没有找到以537395-3813005开头的文件。
没有找到以537348-3812500开头的文件。


In [43]:
df_real.to_excel('real_order.xlsx', index=False, na_rep='<NA>')

In [ ]:
########################################################################

In [44]:
# 读取Excel文件
df_raw = pd.read_excel('course_info_by_summary_and_agg_method_0606_0612.xlsx')

In [45]:
df_raw['course_code_valid'] = np.nan

In [51]:
# 解压zip文件
for index, row in df_raw.iterrows():
    real_file_id = row['file_id']
    if pd.isnull(real_file_id):
        continue
    if '_' in str(real_file_id):
        file_id = int(str(file_id).split('_')[0])
    else:
        file_id = int(real_file_id)
    order_id = int(row['order_id'])
    course_code = df_real.loc[df_real['file_id']==file_id, 'course_code'].iloc[0]
    content = search_file_by_prefix(folder_path, str(order_id)+'-'+str(real_file_id))
    if content is None:
        df_raw.loc[index, 'course_code_valid'] = 'missing'
    elif not content:
        df_raw.loc[index, 'course_code_valid'] = 'blank'
    else:
        if pd.isna(course_code)==False:
            # print(course_code)
            df_raw.loc[index, 'course_code_valid'] = check_nan(course_code, content)

In [32]:
# 不解压zip文件
for index, row in df_raw.iterrows():
    file_id = row['file_id']
    if pd.isnull(file_id):
        continue
    if '_' in str(file_id):
        file_id = str(file_id).split('_')[0]
        df_raw.loc[index, 'course_code_valid'] = 'missing'
    else:
        # print(file_id)
        if file_id in df_real['file_id'].values:
            flag = df_real.loc[df_real['file_id']==file_id, 'course_code_valid'].iloc[0]
            df_raw.loc[index, 'course_code_valid'] = flag

In [52]:
df_raw.to_excel('raw_order.xlsx', index=False, na_rep='<NA>')

In [ ]:
###############################################################################

In [53]:
df_alg = df_raw.iloc[:, 0:7].drop_duplicates()
len(df_alg)

70

In [54]:
human_df = df_real.iloc[:, [0,3,4,5,6,7]].drop_duplicates()
len(human_df)

75

In [55]:
df_output = pd.merge(df_alg, human_df, on=['order_id'], how='left')
new_column_names = {'course_code_x': 'course_code_alg', 'course_name_x': 'course_name_alg', 'university_x': 'university_alg', 'course_code_y': 'course_code_org', 'course_name_y': 'course_name_org', 'university_y': 'uni_org',}
df_output = df_output.rename(columns=new_column_names)

In [56]:
len(df_output)

70

In [57]:
def rouge_similarity(str1, str2):
    if pd.isna(str1) or pd.isna(str2) or str(str1)=='0' or str(str2)=='0':
        return 0
    str1 = str(str1).lower().replace(" ", "")
    str2 = str(str2).lower().replace(" ", "")
    rouge = lawrouge.Rouge()
    scores = rouge.get_scores([str1], [str2], avg=2)
    return scores['f']

In [58]:
df_output['similarity_course_code'] = df_output.apply(lambda row: rouge_similarity(row['course_code_alg'], row['course_code_org']), axis=1)
df_output['similarity_uni'] = df_output.apply(lambda row: rouge_similarity(row['university_alg'], row['uni_org']), axis=1)

In [59]:
select_df1 = df_output[(pd.isna(df_output['course_code_org'])==False) & (df_output['similarity_course_code']<0.9)]
df_output['org_code_valid'] = np.nan

In [60]:
for index, row in select_df1.iterrows():
    # print(index)
    order_id = row['order_id']
    temp_df = list(df_real[df_real['order_id']==order_id]['course_code_valid'])
    if True in temp_df:
        df_output.loc[index, 'org_code_valid'] = 'True'
    else:
        df_output.loc[index, 'org_code_valid'] = 'False'

C:\Users\nlp\AppData\Local\Temp\ipykernel_21640\3253032421.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_output.loc[index, 'org_code_valid'] = 'False'


In [61]:
df_output.to_excel('output.xlsx', index=False, na_rep='<NA>')